## Environment Setup

In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn import preprocessing
import torch.utils.data as Data
from datetime import datetime, timedelta

from radam import RAdam
import data_utils

In [4]:
# Check if PyTorch can use GPU in this environment
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device=='cuda':
    torch.cuda.get_device_name(0)

Torch 1.4.0 CUDA 10.1
Device: cuda:0


In [5]:
# Define random seed for torch
torch.manual_seed(7) # cpu
torch.cuda.manual_seed(7) #gpu

## Parameter Setting

In [6]:
STATION_ID = '001'

In [8]:
"""
Purpose: Merge several one-month train/test data into single file
"""
extension = 'csv'

data_utils.merge_files(STATION_ID,'data/forecast_data/train',STATION_ID+'_train.csv',extension)
data_utils.merge_files(STATION_ID,'data/forecast_data/test',STATION_ID+'_test.csv',extension)

Number of files: 9
Number of files: 3


In [9]:
"""
Load csv data to dataframe
"""
input_train_data = pd.read_csv('data/forecast_data/train/{}_train.csv'.format(STATION_ID), error_bad_lines=False)
input_test_data = pd.read_csv('data/forecast_data/test/{}_test.csv'.format(STATION_ID), error_bad_lines=False)

observation_data = pd.read_csv('data/observation_data/{}.csv'.format(STATION_ID), error_bad_lines=False)

print('input_train_data Shape: ', input_train_data.shape)
print('input_test_data Shape: ', input_test_data.shape)

input_train_data.head()

input_train_data Shape: (102363, 33)
input_test_data Shape: (33968, 33)


,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,RH01,...,DSWRF,HCDC,LCDC,MCDC,PRES,PRMSL,RH,TMP,UGRD,VGRD
0,1,2017010120,2017,1,1,20,646.3,3192.7,4.9,15,...,0.0,0.0,0.000000,0.000000,36553.144444,102715.444444,90.333333,281.022222,-1.908889,0.995556
1,1,2017010121,2017,1,1,21,646.5,3193.5,5.4,14,...,0.0,0.0,5.333333,1.333333,36538.138889,102515.333333,56.677778,278.113333,1.588889,-0.678889
2,1,2017010122,2017,1,1,22,646.7,3192.7,5.1,14,...,0.0,0.0,6.666667,2.111111,36524.805556,102486.222222,50.922222,278.266667,0.513333,-0.097778
3,1,2017010123,2017,1,1,23,646.6,3188.9,5.6,14,...,0.0,0.0,3.777778,1.444444,36520.372222,102456.888889,45.311111,278.201111,0.988889,-0.543333
4,1,2017010124,2017,1,1,24,646.5,3190.3,5.2,14,...,0.0,0.0,1.777778,0.222222,36531.083333,102467.888889,40.944444,278.458889,0.711111,-0.532222


In [10]:
"""
Clean the data
  -9991 儀器故障待修無資料,
  -9996 資料累計於後,
  -9997 因不明原因或故障等因素無資料,
  -9998 雨跡(Trace),
  -9999 未觀測而無資料
"""
input_train_data = input_train_data[input_train_data.TX01>-9991]
input_test_data = input_test_data[input_test_data.TX01>-9991]

print('input_train_data Shape: ' + str(input_train_data.shape))
print('input_test_data Shape: ' + str(input_test_data.shape))

input_train_data Shape: (100729, 33)
input_test_data Shape: (33968, 33)


新增時間欄位

由於時間有週期性，因此用 sin 跟 cos 來表示

參考：https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/

In [11]:
hours_in_day = 24

input_train_data_hour_in_a_day_radian = 2*np.pi*input_train_data.hour/hours_in_day
input_train_data['sin_time'] = np.sin(input_train_data_hour_in_a_day_radian)
input_train_data['cos_time'] = np.cos(input_train_data_hour_in_a_day_radian)

input_test_data_hour_in_a_day_radian = 2*np.pi*input_test_data.hour/hours_in_day
input_test_data['sin_time'] = np.sin(input_test_data_hour_in_a_day_radian)
input_test_data['cos_time'] = np.cos(input_test_data_hour_in_a_day_radian)

months_in_year = 12
input_train_data_month_in_a_year_radian = 2*np.pi*input_train_data.month/months_in_year
input_train_data['sin_month_time'] = np.sin(input_train_data_month_in_a_year_radian)
input_train_data['cos_month_time'] = np.cos(input_train_data_month_in_a_year_radian)

input_test_data_month_in_a_year_radian = 2*np.pi*input_test_data.month/months_in_year
input_test_data['sin_month_time'] = np.sin(input_test_data_month_in_a_year_radian)
input_test_data['cos_month_time'] = np.cos(input_test_data_month_in_a_year_radian)

input_test_data.head()

,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,RH01,...,PRES,PRMSL,RH,TMP,UGRD,VGRD,sin_time,cos_time,sin_month_time,cos_month_time
0,1,2017100108,2017,10,1,8,647.8,3206.0,11.0,70,...,73214.455556,102470.333333,71.900000,289.585556,-1.446667,0.693333,8.660254e-01,-0.500000,-0.866025,0.5
1,1,2017100109,2017,10,1,9,648.3,3210.7,13.0,62,...,36617.755556,102443.555556,78.000000,287.600000,-0.601111,0.803333,7.071068e-01,-0.707107,-0.866025,0.5
2,1,2017100110,2017,10,1,10,648.3,3209.0,14.6,58,...,36610.172222,102401.888889,77.611111,287.945556,-0.185556,0.483333,5.000000e-01,-0.866025,-0.866025,0.5
3,1,2017100111,2017,10,1,11,647.9,3201.5,16.6,54,...,36620.750000,102427.222222,83.644444,287.871111,0.108889,-0.243333,2.588190e-01,-0.965926,-0.866025,0.5
4,1,2017100112,2017,10,1,12,647.2,3191.0,17.0,54,...,36619.077778,102413.444444,89.566667,287.857778,-0.034444,-0.592222,1.224647e-16,-1.000000,-0.866025,0.5


產生時雨量
將前後兩小時的累積雨量相減

In [12]:
start_times = input_train_data['start_time'].unique()
str2date = {}
for start_time in start_times:
    start_time_date = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S %Z')
    start_time_date_prev = start_time_date + timedelta(hours=-1)
    str2date[start_time] = "{} UTC".format(start_time_date_prev)

input_train_data['start_time_prev'] = input_train_data.apply(lambda x: str2date[x['start_time']] , axis=1)

start_times = input_test_data['start_time'].unique()
str2date = {}
for start_time in start_times:
    start_time_date = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S %Z')
    start_time_date_prev = start_time_date + timedelta(hours=-1)
    str2date[start_time] = "{} UTC".format(start_time_date_prev)
input_test_data['start_time_prev'] = input_test_data.apply(lambda x: str2date[x['start_time']] , axis=1)

In [13]:
df_byhour_prev = input_train_data[['start_time','issue_time','APCP']].copy()
df_byhour_prev.rename(columns={'start_time':'start_time_prev','APCP':'APCP_prev'},inplace=True)
df_byhour2 = pd.merge(input_train_data, df_byhour_prev, how='left', on=['start_time_prev','issue_time'])

df_byhour2['APCP_diff'] = df_byhour2['APCP'] - df_byhour2['APCP_prev']
input_train_data = df_byhour2

df_byhour_prev = input_test_data[['start_time','issue_time','APCP']].copy()
df_byhour_prev.rename(columns={'start_time':'start_time_prev','APCP':'APCP_prev'},inplace=True)
df_byhour2 = pd.merge(input_test_data, df_byhour_prev, how='left', on=['start_time_prev','issue_time'])

df_byhour2['APCP_diff'] = df_byhour2['APCP'] - df_byhour2['APCP_prev']
input_test_data = df_byhour2

C:\Users\jason.hsu\.conda\envs\aigo-weather\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
input_train_data['APCP_diff'] = input_train_data.apply(lambda row: data_utils.remove_null_column(row, 'APCP_diff'), axis=1)
input_test_data['APCP_diff'] = input_test_data.apply(lambda row: data_utils.remove_null_column(row, 'APCP_diff'), axis=1)

In [15]:
input_train_data['tmp_diff'] = input_train_data['TX01'] - (input_train_data['TMP']-273.15)
input_test_data['tmp_diff'] = input_test_data['TX01'] - (input_test_data['TMP']-273.15)

In [16]:
input_test_data = input_test_data.sort_values(['yyyymmddhh','number_of_hour'], ascending=[True,True])
input_test_data.head()

,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,RH01,...,UGRD,VGRD,sin_time,cos_time,sin_month_time,cos_month_time,start_time_prev,APCP_prev,APCP_diff,tmp_diff
0,1,2017100108,2017,10,1,8,647.8,3206.0,11.0,70,...,-1.446667,0.693333,8.660254e-01,-0.500000,-0.866025,0.5,2017-09-30 23:00:00 UTC,NaN,0.000000,-5.435556
1,1,2017100109,2017,10,1,9,648.3,3210.7,13.0,62,...,-0.601111,0.803333,7.071068e-01,-0.707107,-0.866025,0.5,2017-10-01 00:00:00 UTC,0.000000,0.000000,-1.450000
2,1,2017100110,2017,10,1,10,648.3,3209.0,14.6,58,...,-0.185556,0.483333,5.000000e-01,-0.866025,-0.866025,0.5,2017-10-01 01:00:00 UTC,0.000000,0.000000,-0.195556
3,1,2017100111,2017,10,1,11,647.9,3201.5,16.6,54,...,0.108889,-0.243333,2.588190e-01,-0.965926,-0.866025,0.5,2017-10-01 02:00:00 UTC,0.000000,0.004111,1.878889
4,1,2017100112,2017,10,1,12,647.2,3191.0,17.0,54,...,-0.034444,-0.592222,1.224647e-16,-1.000000,-0.866025,0.5,2017-10-01 03:00:00 UTC,0.004111,0.163667,2.292222


Prepare Data

Clean the data

*  -9991 儀器故障待修無資料,
*  -9996 資料累計於後,
*  -9997 因不明原因或故障等因素無資料,
*  -9998 雨跡(Trace),
*  -9999 未觀測而無資料

資料狀況
*  CD11 太多 -9999
*  SS01 -9999 有7706筆
*  SS02 -9999/-9997 有662筆
*  PP01 -9999/-9997 有581筆

整理資料
*  TX01：清除
*  RH01：→100
*  PP01：→0
*  PS01：→643.463(平均值)
*  PS02：→3159.6909(平均值)
*  WD01：→0
*  WD02：→0



In [17]:
observation_data.head()

,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,TX04,...,ST06,ST07,ST08,ST09,ST10,ST11,ST12,PS11,RH06,UV01
0,1,2017010101,2017,1,1,1,646.3,3192.7,3.4,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0
1,1,2017010102,2017,1,1,2,646.1,3190.1,3.3,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0
2,1,2017010103,2017,1,1,3,645.9,3187.0,3.1,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0
3,1,2017010104,2017,1,1,4,645.6,3184.8,3.2,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0
4,1,2017010105,2017,1,1,5,645.8,3190.2,3.3,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0


In [18]:
"""
Clean the data
  -9991 儀器故障待修無資料,
  -9996 資料累計於後,
  -9997 因不明原因或故障等因素無資料,
  -9998 雨跡(Trace),
  -9999 未觀測而無資料
"""
observation_data['observation_RH01'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'RH01',100), axis=1)
observation_data['observation_PP01'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'PP01',0), axis=1)
observation_data['observation_PS01'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'PS01',643.463), axis=1)
observation_data['observation_PS02'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'PS02',3159.6909), axis=1)
observation_data['observation_WD01'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'WD01',0), axis=1)
observation_data['observation_WD02'] = observation_data.apply(lambda row: data_utils.arrange_abnormal_data(row,'WD02',0), axis=1)

新增時間欄位

由於時間有週期性，因此用 sin 跟 cos 來表示

In [19]:
seconds_in_day = 24
months_in_year = 12

observation_data_hour_in_a_day_radian = 2*np.pi*observation_data.hour/hours_in_day
observation_data['observation_sin_time'] = np.sin(observation_data_hour_in_a_day_radian)
observation_data['observation_cos_time'] = np.cos(observation_data_hour_in_a_day_radian)

observation_data_month_in_a_year_radian = 2*np.pi*observation_data.month/months_in_year
observation_data['observation_data_sin_month_time'] = np.sin(observation_data_month_in_a_year_radian)
observation_data['observation_data_cos_month_time'] = np.cos(observation_data_month_in_a_year_radian)

In [20]:
observation_data.head()

,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,TX04,...,observation_RH01,observation_PP01,observation_PS01,observation_PS02,observation_WD01,observation_WD02,observation_sin_time,observation_cos_time,observation_data_sin_month_time,observation_data_cos_month_time
0,1,2017010101,2017,1,1,1,646.3,3192.7,3.4,-9999,...,15.0,0.0,646.3,3192.7,5.3,340.0,0.258819,0.965926,0.5,0.866025
1,1,2017010102,2017,1,1,2,646.1,3190.1,3.3,-9999,...,15.0,0.0,646.1,3190.1,4.4,320.0,0.500000,0.866025,0.5,0.866025
2,1,2017010103,2017,1,1,3,645.9,3187.0,3.1,-9999,...,15.0,0.0,645.9,3187.0,4.5,320.0,0.707107,0.707107,0.5,0.866025
3,1,2017010104,2017,1,1,4,645.6,3184.8,3.2,-9999,...,15.0,0.0,645.6,3184.8,4.8,330.0,0.866025,0.500000,0.5,0.866025
4,1,2017010105,2017,1,1,5,645.8,3190.2,3.3,-9999,...,15.0,0.0,645.8,3190.2,6.1,340.0,0.965926,0.258819,0.5,0.866025


In [21]:
input_train_data.head()

,stno,yyyymmddhh,year,month,day,hour,PS01,PS02,TX01,RH01,...,UGRD,VGRD,sin_time,cos_time,sin_month_time,cos_month_time,start_time_prev,APCP_prev,APCP_diff,tmp_diff
0,1,2017010120,2017,1,1,20,646.3,3192.7,4.9,15,...,-1.908889,0.995556,-8.660254e-01,0.500000,0.5,0.866025,2017-01-01 11:00:00 UTC,NaN,0.0,-2.972222
1,1,2017010121,2017,1,1,21,646.5,3193.5,5.4,14,...,1.588889,-0.678889,-7.071068e-01,0.707107,0.5,0.866025,2017-01-01 12:00:00 UTC,0.0,0.0,0.436667
2,1,2017010122,2017,1,1,22,646.7,3192.7,5.1,14,...,0.513333,-0.097778,-5.000000e-01,0.866025,0.5,0.866025,2017-01-01 13:00:00 UTC,0.0,0.0,-0.016667
3,1,2017010123,2017,1,1,23,646.6,3188.9,5.6,14,...,0.988889,-0.543333,-2.588190e-01,0.965926,0.5,0.866025,2017-01-01 14:00:00 UTC,0.0,0.0,0.548889
4,1,2017010124,2017,1,1,24,646.5,3190.3,5.2,14,...,0.711111,-0.532222,-2.449294e-16,1.000000,0.5,0.866025,2017-01-01 15:00:00 UTC,0.0,0.0,-0.108889


準備資料

將觀測資料的指定時間區段納入考量

In [22]:
TIME_SHIFT = 120+193
observation_data['shifted_start_time'] = observation_data.apply(lambda row: data_utils.generate_time_shift_columns(row,TIME_SHIFT), axis=1)
observation_data.rename(columns={'yyyymmddhh': 'observation_yyyymmddhh',
'TX01': 'observation_TX01','RH01': 'unused_RH01',
'PP01': 'unused_PP01','WD01': 'unused_WD01',
'WD02': 'unused_WD02','PS01': 'unused_PS01',
'PS02': 'unused_PS02','CD11': 'observation_CD11',
'SS01': 'observation_SS01','SS02': 'observation_SS02',
}, inplace=True)

observation_data.head()

,stno,observation_yyyymmddhh,year,month,day,hour,unused_PS01,unused_PS02,observation_TX01,TX04,...,observation_PP01,observation_PS01,observation_PS02,observation_WD01,observation_WD02,observation_sin_time,observation_cos_time,observation_data_sin_month_time,observation_data_cos_month_time,shifted_start_time
0,1,2017010101,2017,1,1,1,646.3,3192.7,3.4,-9999,...,0.0,646.3,3192.7,5.3,340.0,0.258819,0.965926,0.5,0.866025,2016-12-19 00:00:00 UTC
1,1,2017010102,2017,1,1,2,646.1,3190.1,3.3,-9999,...,0.0,646.1,3190.1,4.4,320.0,0.500000,0.866025,0.5,0.866025,2016-12-19 01:00:00 UTC
2,1,2017010103,2017,1,1,3,645.9,3187.0,3.1,-9999,...,0.0,645.9,3187.0,4.5,320.0,0.707107,0.707107,0.5,0.866025,2016-12-19 02:00:00 UTC
3,1,2017010104,2017,1,1,4,645.6,3184.8,3.2,-9999,...,0.0,645.6,3184.8,4.8,330.0,0.866025,0.500000,0.5,0.866025,2016-12-19 03:00:00 UTC
4,1,2017010105,2017,1,1,5,645.8,3190.2,3.3,-9999,...,0.0,645.8,3190.2,6.1,340.0,0.965926,0.258819,0.5,0.866025,2016-12-19 04:00:00 UTC


In [23]:
input_train_data['shifted_start_time'] = input_train_data['start_time']
input_test_data['shifted_start_time'] = input_test_data['start_time']

input_train_data2 = data_utils.join(input_train_data,observation_data,'input_train_data2.csv',False)
input_test_data2 = data_utils.join(input_test_data,observation_data,'input_test_data2.csv',False)

input_train_data2.head()

,stno_x,yyyymmddhh,year_x,month_x,day_x,hour_x,PS01,PS02,TX01,RH01,...,observation_RH01,observation_PP01,observation_PS01,observation_PS02,observation_WD01,observation_WD02,observation_sin_time,observation_cos_time,observation_data_sin_month_time,observation_data_cos_month_time
0,1,2017010120,2017,1,1,20,646.3,3192.7,4.9,15,...,100.0,0.5,639.5,3120.5,5.4,300.0,-0.258819,-0.965926,0.5,0.866025
1,1,2017010121,2017,1,1,21,646.5,3193.5,5.4,14,...,98.0,0.0,639.6,3121.8,6.8,350.0,-0.500000,-0.866025,0.5,0.866025
2,1,2017010122,2017,1,1,22,646.7,3192.7,5.1,14,...,98.0,0.0,639.2,3116.7,6.8,340.0,-0.707107,-0.707107,0.5,0.866025
3,1,2017010123,2017,1,1,23,646.6,3188.9,5.6,14,...,96.0,0.0,639.6,3123.1,8.5,340.0,-0.866025,-0.500000,0.5,0.866025
4,1,2017010124,2017,1,1,24,646.5,3190.3,5.2,14,...,98.0,0.0,640.0,3130.2,6.2,340.0,-0.965926,-0.258819,0.5,0.866025


Input Normalization

In [24]:
input_train_data2 = input_train_data
input_test_data2 = input_test_data

In [25]:
%%time
input_forecast_column_names = ['PRMSL','PRES','TMP','RH','UGRD','VGRD','APCP_diff','LCDC','MCDC','HCDC','DSWRF','sin_time',
                               'cos_time','sin_month_time','cos_month_time']
input_observation_column_names = ['observation_TX01','observation_RH01','observation_PP01','observation_PS01',
        'observation_PS02','observation_WD01','observation_WD02','observation_sin_time','observation_cos_time',
        'observation_data_sin_month_time','observation_data_cos_month_time']
column_names = input_forecast_column_names
output_column_names = ['tmp_diff']
group_column_name = ['issue_time']
all_column_names = column_names + output_column_names + group_column_name + ['original_TMP']

input_train_data2['original_TMP'] = input_train_data2['TMP']
input_test_data2['original_TMP'] = input_test_data2['TMP']

NORMALIZE_FLAG = True
SCALAR_TYPE = 1
COLUMNS_TO_NORMALIZE = ['PRMSL','PRES','TMP','RH','UGRD','VGRD','APCP_diff','LCDC','MCDC','HCDC','DSWRF']

train_df = input_train_data2[all_column_names].copy()
test_df = input_test_data2[all_column_names].copy()

input_train_data_mean = train_df.mean()
input_train_data_std = train_df.std()

# Create scaler for normalization
if SCALAR_TYPE == 0:
    scaler = preprocessing.MinMaxScaler()
elif SCALAR_TYPE == 1:
    scaler = preprocessing.StandardScaler()

if NORMALIZE_FLAG:
    # Scale preprocessing only apply to the specified columns
    train_df[COLUMNS_TO_NORMALIZE] = scaler.fit_transform(train_df[COLUMNS_TO_NORMALIZE])

    train_normalized = train_df.values
else:
    train_normalized = train_df.values
    test_normalized = test_df.values

C:\Users\jason.hsu\.conda\envs\aigo-weather\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jason.hsu\.conda\envs\aigo-weather\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 5min 37s


Test dataset's head info (Before Normalization)

In [26]:
test_df.head()

,PRMSL,PRES,TMP,RH,UGRD,VGRD,APCP_diff,LCDC,MCDC,HCDC,DSWRF,sin_time,cos_time,sin_month_time,cos_month_time,tmp_diff,issue_time,original_TMP
0,102470.333333,73214.455556,289.585556,71.900000,-1.446667,0.693333,0.000000,0.000000,0.000000,0.000000,0.000000,8.660254e-01,-0.500000,-0.866025,0.5,-5.435556,2017-10-01 00:00:00 UTC,289.585556
1,102443.555556,36617.755556,287.600000,78.000000,-0.601111,0.803333,0.000000,0.000000,0.111111,28.000000,683.722222,7.071068e-01,-0.707107,-0.866025,0.5,-1.450000,2017-10-01 00:00:00 UTC,287.600000
2,102401.888889,36610.172222,287.945556,77.611111,-0.185556,0.483333,0.000000,4.933333,0.000000,25.888889,860.633333,5.000000e-01,-0.866025,-0.866025,0.5,-0.195556,2017-10-01 00:00:00 UTC,287.945556
3,102427.222222,36620.750000,287.871111,83.644444,0.108889,-0.243333,0.004111,15.877778,3.000000,26.222222,886.111111,2.588190e-01,-0.965926,-0.866025,0.5,1.878889,2017-10-01 00:00:00 UTC,287.871111
4,102413.444444,36619.077778,287.857778,89.566667,-0.034444,-0.592222,0.163667,20.866667,8.666667,41.888889,776.222222,1.224647e-16,-1.000000,-0.866025,0.5,2.292222,2017-10-01 00:00:00 UTC,287.857778


In [27]:
print(input_train_data_mean)

PRMSL             102149.635732
PRES               36494.089692
TMP                  282.461429
RH                    78.160661
UGRD                   1.908172
VGRD                   0.821651
APCP_diff              0.667071
LCDC                  15.746791
MCDC                  16.963122
HCDC                  38.587584
DSWRF                240.494503
sin_time               0.000093
cos_time              -0.000005
sin_month_time         0.128990
cos_month_time        -0.256872
tmp_diff              -4.222787
original_TMP         282.461429
dtype: float64


In [28]:
print(input_train_data_std)

PRMSL              347.412803
PRES              1540.242015
TMP                  4.134049
RH                  17.057335
UGRD                 2.943750
VGRD                 2.014284
APCP_diff            2.622685
LCDC                18.570541
MCDC                25.130392
HCDC                38.982537
DSWRF              325.711011
sin_time             0.708008
cos_time             0.706212
sin_month_time       0.744201
cos_month_time       0.602952
tmp_diff             2.467783
original_TMP         4.134049
dtype: float64


In [29]:
# Apply the training data's mean and std to normalize testing data
if SCALAR_TYPE == 1:
    for column_name in COLUMNS_TO_NORMALIZE:
        index = all_column_names.index(column_name)

        test_df[column_name] = test_df.apply(lambda row: data_utils.standard_normalization(row,column_name,index,
           input_train_data_mean,input_train_data_std), axis=1)

C:\Users\jason.hsu\.conda\envs\aigo-weather\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Test dataset's head info (After Normalization)

In [30]:
test_df.head()

,PRMSL,PRES,TMP,RH,UGRD,VGRD,APCP_diff,LCDC,MCDC,HCDC,DSWRF,sin_time,cos_time,sin_month_time,cos_month_time,tmp_diff,issue_time,original_TMP
0,0.923102,23.840647,1.723281,-0.367036,-1.139648,-0.063704,-0.254347,-0.847945,-0.675004,-0.989868,-0.738368,8.660254e-01,-0.500000,-0.866025,0.5,-5.435556,2017-10-01 00:00:00 UTC,289.585556
1,0.846025,0.080290,1.242987,-0.009419,-0.852410,-0.009094,-0.254347,-0.847945,-0.670583,-0.271598,1.360801,7.071068e-01,-0.707107,-0.866025,0.5,-1.450000,2017-10-01 00:00:00 UTC,287.600000
2,0.726091,0.075366,1.326575,-0.032218,-0.711245,-0.167959,-0.254347,-0.582291,-0.675004,-0.325753,1.903954,5.000000e-01,-0.866025,-0.866025,0.5,-0.195556,2017-10-01 00:00:00 UTC,287.945556
3,0.799011,0.082234,1.308567,0.321491,-0.611221,-0.528716,-0.252779,0.007053,-0.555627,-0.317203,1.982176,2.588190e-01,-0.965926,-0.866025,0.5,1.878889,2017-10-01 00:00:00 UTC,287.871111
4,0.759352,0.081148,1.305342,0.668686,-0.659912,-0.701924,-0.191942,0.275699,-0.330136,0.084687,1.644795,1.224647e-16,-1.000000,-0.866025,0.5,2.292222,2017-10-01 00:00:00 UTC,287.857778


用 issue_time 來做 group by 以產生每組 sequence data

In [31]:
train_groupdf = train_df.groupby("issue_time")
test_groupdf = test_df.groupby("issue_time")

In [32]:
train_issue_times = input_train_data['issue_time'].unique()
test_issue_times = input_test_data['issue_time'].unique()

In [33]:
train_input_keys = train_groupdf.groups.keys()
train_inputs,train_outputs = data_utils.get_io_data(train_input_keys,train_groupdf,column_names+['original_TMP'],output_column_names,False)

test_input_keys = test_groupdf.groups.keys()
test_inputs,test_outputs = data_utils.get_io_data(test_input_keys,test_groupdf,column_names+['original_TMP'],output_column_names,False)

## LSTM
[Ref](https://morvanzhou.github.io/tutorials/machine-learning/torch/4-02-RNN-classification/)

[Ref2](https://blog.csdn.net/qq_26657001/article/details/87892833)

In [34]:
# Hyper Parameters
TIME_STEP = 193      # rnn time step
INPUT_SIZE = len(column_names)      # rnn input size

In [35]:
class LSTMRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_of_layers):
        super(LSTMRNN, self).__init__()

        self.num_of_layers = num_of_layers
        self.hidden_size = hidden_size
        self.rnn = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_of_layers, 
            batch_first=True,
            dropout=0.2,
        )
        self.out = torch.nn.Linear(hidden_size, 1)

    def init_hidden(self, batch_size):
        h0=torch.randn(self.num_of_layers,batch_size,self.hidden_size)
        c0=torch.randn(self.num_of_layers,batch_size,self.hidden_size)
        return (h0,c0)
    
    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        r_out = r_out.contiguous().view(-1, self.hidden_size)
        outs = self.out(r_out)
        outs = outs.view(-1, TIME_STEP, 1)
        return outs, h_state

Data Preparation

In [36]:
HIDDEN_SIZE = 128
NUM_OF_LAYERS = 3
HIDDEN_SIZE_2 = 32
NUM_OF_LAYERS_2 = 1
LR = 0.001
h_state = None
BATCH_SIZE = 64
TEST_BATCH_SIZE = 64
INPUT_SIZE = len(input_forecast_column_names)
batch_decoder_x_len = len(input_forecast_column_names)
EPOCHS = 36
MAX_STEP = 100

rnn = LSTMRNN(INPUT_SIZE,HIDDEN_SIZE,NUM_OF_LAYERS)

# Create optimizer and loss function
optimizer = RAdam(rnn.parameters(), lr=LR)
loss_func = torch.nn.MSELoss()

In [37]:
# Convert (batch_size, seq_len, input_size) to (seq_len, batch_size, input_size)
# Convert numpy to tensor
train_input_tensors = torch.FloatTensor(train_inputs)
test_input_tensors = torch.FloatTensor(test_inputs)

# 2 sequences (to match the batch size) of length 6 (for the 6h into the future)
# Convert numpy to tensor
train_output_tensors = torch.FloatTensor(train_outputs)
test_output_tensors = torch.FloatTensor(test_outputs)

# 先轉換 torch 能識別的 Dataset
train_dataset = Data.TensorDataset(train_input_tensors,train_output_tensors)
test_dataset = Data.TensorDataset(test_input_tensors,test_output_tensors)

# After creating the workers, each worker has an independent seed that is initialized to the curent random seed + the id of the worker
def worker_init_fn(worker_id):
    np.random.seed(7)

# 把 dataset 放入 DataLoader
train_dataloader = Data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,           # 要不要打亂數據
    worker_init_fn=worker_init_fn   # random seed
)

test_dataloader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    worker_init_fn=worker_init_fn
)

In [38]:
%%time
for epoch in range(EPOCHS):
    for step, (batch_full_x, batch_y) in enumerate(train_dataloader):  # 每一步 loader 釋放一小批數據來學習
        rnn.train()
        optimizer.zero_grad()  # clear gradients for this training step

        # (batch, seq_len, input_size)
        batch_x = batch_full_x[:,:,:batch_decoder_x_len]

        if step > MAX_STEP:
            break

        # If the last batch from DataLoader is less than the BATCH_SIZE,
        # then it couldn't be fed into the model or there would be an error
        if list(batch_x.size())[0] != BATCH_SIZE:
            break

        # Reset hidden state of encoder for current batch
        h_state = rnn.init_hidden(batch_x.shape[0])
        
        prediction, h_state = rnn(batch_x, h_state)

        loss = loss_func(prediction, batch_y)
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients

        print('Epoch: ', epoch, '| Step: ', step, "| Training Loss(RMSE): ", loss.item()**0.5)

    total_test_loss = 0
    number_of_batch = 0

    rnn.eval()
    
    for i, (batch_full_x, batch_y) in enumerate(test_dataloader):

        batch_x = batch_full_x[:,:,:batch_decoder_x_len]

        # If the last batch from DataLoader is less than the TEST_BATCH_SIZE,
        # then it couldn't be fed into the model or there would be an error
        if list(batch_x.size())[0] != TEST_BATCH_SIZE:
            break

        # Reset hidden state of encoder for current batch
        h_state = rnn.init_hidden(batch_x.shape[0])
        
        prediction, h_state = rnn(batch_x, h_state)

        loss = loss_func(prediction, batch_y)

        total_test_loss += loss.item()
        number_of_batch = i
        
    number_of_batch += 1
    test_rmse = (total_test_loss/number_of_batch)**0.5
    print ("Epoch: ", epoch, "| Test Loss (RMSE): ", test_rmse)    

Epoch:  0 | Step:  0 | Training Loss(RMSE):  4.873499908487967
Epoch:  0 | Step:  1 | Training Loss(RMSE):  4.952523083620997
Epoch:  0 | Step:  2 | Training Loss(RMSE):  4.856286542403938
Epoch:  0 | Step:  3 | Training Loss(RMSE):  5.018276951444726
Epoch:  0 | Step:  4 | Training Loss(RMSE):  4.895133673271775
Epoch:  0 | Step:  5 | Training Loss(RMSE):  4.771035846704268
Epoch:  0 | Step:  6 | Training Loss(RMSE):  4.777622217903676
Epoch:  0 | Test Loss (RMSE):  4.432373847005316
Epoch:  1 | Step:  0 | Training Loss(RMSE):  4.899262719252841
Epoch:  1 | Step:  1 | Training Loss(RMSE):  4.78407792378119
Epoch:  1 | Step:  2 | Training Loss(RMSE):  4.847050483719314
Epoch:  1 | Step:  3 | Training Loss(RMSE):  4.835598491563009
Epoch:  1 | Step:  4 | Training Loss(RMSE):  4.816699091044301
Epoch:  1 | Step:  5 | Training Loss(RMSE):  4.978584393834087
Epoch:  1 | Step:  6 | Training Loss(RMSE):  4.961643150785565
Epoch:  1 | Test Loss (RMSE):  4.419434969417875
Epoch:  2 | Step:  0 